tkmoddbg or tool.mod

Easiest way: turn it into a VS Code snippet so typing something like `dmdbg` auto-suggests it and Tab drops it in.

I’ll assume you’re using VS Code / Windsurf (same snippet system). Here’s how to do it.

---

## 1. Open your Python snippets file

1. Press **Cmd+Shift+P**
2. Type **“Configure User Snippets”** → hit Enter
3. Choose **`python`** (or “New Global Snippets file” if you prefer)

That opens a JSON file like `python.json`.

---

## 2. Add a snippet for `debug_module` helper

In that JSON, add something like this (inside the outer `{ ... }`):

```jsonc
{
  "debug_module_helper": {
    "prefix": "dmdbg",
    "description": "Debug + reload a module with optional smoke tests",
    "body": [
      "from importlib import reload",
      "import inspect",
      "",
      "def debug_module(mod, test_funcs=None):",
      "    \"\"\"",
      "    Quick helper to:",
      "      - reload a module",
      "      - print where it's loaded from",
      "      - optionally execute some test functions",
      "    \"\"\"",
      "    mod = reload(mod)",
      "    print(\"✅ Reloaded:\", mod.__name__)",
      "    print(\"   File   :\", getattr(mod, \"__file__\", \"<no file>\"))",
      "",
      "    if test_funcs:",
      "        for name, fn in test_funcs.items():",
      "            try:",
      "                print(f\"\\n— Running smoke test: {name}()\")",
      "                fn()",
      "                print(f\"   ✔ {name} ok\")",
      "            except Exception as e:",
      "                print(f\"   ⚠ {name} failed: {e}\")",
      "",
      "    return mod",
      ""
    ]
  }
}
```

Now in any **`.py`** file:

1. Type `dmdbg`
2. IntelliSense will suggest **`debug_module_helper`**
3. Hit **Tab** → full helper drops in.

---

## 3. Optional: snippet for the *usage* pattern

You can also have a snippet that inserts the “smoke-test a module” pattern:

```jsonc
"debug_module_usage": {
  "prefix": "dmuse",
  "description": "Use debug_module to reload + smoke-test a module",
  "body": [
    "import ${1:telco_churn.utils.reporting} as mod_under_test",
    "from toolkit.module_debug import debug_module",
    "",
    "def _test_${2:log_section}():",
    "    mod_under_test.${2:log_section_completion}(\"X.Y.Z\", \"OK\", checked=1, mismatched=0)",
    "",
    "mod_under_test = debug_module(",
    "    mod_under_test,",
    "    test_funcs={\"${2:log_section}\": _test_${2:log_section}},",
    ")"
  ]
}
```

Then:

* Type `dmuse` → Tab → you get the usage skeleton
* You can Tab through `${1:...}` and `${2:...}` to fill in module / function names.

---

## 4. Still keep it in `toolkit/module_debug.py`

You’re doing the right thing by **also** having:

```python
# toolkit/module_debug.py
from importlib import reload
import inspect

def debug_module(mod, test_funcs=None):
    ...
```

The snippet is just to *write* that file (or reuse the pattern) quickly in future projects. The actual reusable logic lives in your toolkit repo.

If you tell me your exact prefix preference (like `tkmoddbg` or `tool.mod`), I can tune the snippet names to match your mental map.
'

Nice, you’re thinking about this exactly like a real “dev environment” owner. 😄

There are basically **two good ways** to make your whole toolkit feel auto-suggested:

---

## 1. Project-wide snippets (per-project “toolkit header”)

Instead of a snippet for *one* helper, you create a **workspace snippet file** that:

* lives in the Telco repo
* only applies when you’re in that project
* can expand into:

  * common `import` lines for your toolkit modules
  * optional “starter usage” patterns

### Step 1 — Create a workspace snippet file

In VS Code / Windsurf:

1. **Cmd+Shift+P** → “**Configure User Snippets**”
2. Choose **“New Snippets file for ‘Workspace’…”**
3. Name it something like: `telco-toolkit.code-snippets`

That file will live in your `.vscode/` folder and only apply to this project.

### Step 2 — Add a “toolkit header” snippet

Inside that JSON file, you can define something like:

```jsonc
{
  "Telco Toolkit Header": {
    "prefix": "tkhdr",
    "description": "Import the Telco toolkit + common debug utilities",
    "body": [
      "# Telco toolkit imports",
      "from toolkit.module_debug import debug_module",
      "from telco_churn.section2.reporting import append_sec2, log_section_completion",
      "from telco_churn.utils.metrics_utils import summarize_append_refactor",
      "",
      "# Example smoke-test pattern",
      "def _test_section_logger():",
      "    log_section_completion(",
      "        \"X.Y.Z\",",
      "        \"OK\",",
      "        checked=1,",
      "        mismatched=0,",
      "    )",
      "",
      "# Example module debug use",
      "if __name__ == \"__main__\":",
      "    import telco_churn.section2.reporting as reporting",
      "    reporting = debug_module(",
      "        reporting,",
      "        test_funcs={\"log_section_completion\": _test_section_logger},",
      "    )",
      ""
    ]
  }
}
```

Now anywhere in this repo:

* Type `tkhdr`
* Hit **Tab**
* And you instantly get your “toolkit starter pack” in that file.

You can add **more snippets** in the same file, e.g.:

* `tks2` → skeleton for a new Section 2 cell
* `tkrun` → standard `if __name__ == "__main__":` runner that wires in logging, run_id, etc.
* `tklog` → quick `log_section_completion` usage block

---

## 2. Make the toolkit itself “one import away”

Instead of inserting the *whole toolkit code* via snippets (which gets messy), keep the code in real modules and let snippets mainly handle:

* the **import lines**
* common **usage patterns**

Example: create a **`telco_churn/toolkit/__init__.py`** that re-exports your core helpers:

```python
# telco_churn/toolkit/__init__.py

from .module_debug import debug_module
from telco_churn.section2.reporting import append_sec2, log_section_completion
from telco_churn.utils.metrics_utils import summarize_append_refactor

__all__ = [
    "debug_module",
    "append_sec2",
    "log_section_completion",
    "summarize_append_refactor",
]
```

Then in your notebooks/scripts you can:

```python
from telco_churn import toolkit as tk

tk.log_section_completion("2.1.5", "OK", checked=21, mismatched=0)
tk.debug_module(...)
```

Now your **editor’s autocomplete** becomes the “auto-suggest the toolkit” mechanism:

* Type `tk.` → see `debug_module`, `append_sec2`, etc.

You can combine this with a tiny snippet:

```jsonc
"Import Telco toolkit": {
  "prefix": "tktk",
  "description": "Import Telco toolkit namespace",
  "body": [
    "from telco_churn import toolkit as tk",
    ""
  ]
}
```

Type `tktk` → Tab → now `tk.` gives you everything.

---

## 3. Which approach where?

* **Workspace snippets** → great for:

  * “starter” files
  * repeating patterns (new section, new module, debug harness)
  * quickly dropping in a standard header

* **Toolkit package (`telco_churn.toolkit`)** → great for:

  * reusability across notebooks, scripts, future projects
  * leveraging your IDE’s normal completion (`tk.`)
  * keeping actual logic in one place instead of copy-pasted snippets

If you want, I can sketch a **minimal `telco_churn/toolkit` layout** (files + content) that matches what you already have (`debug_module`, `log_section_completion`, etc.) and a matching set of 2–3 high-value snippets (`tkhdr`, `tktk`, `tks2`).

Nice, you’re *officially* in “I have my own dev toolkit” territory now. 🔧😄

Your snippet file is **very close**, but there are a couple of issues that will break it:

### Main problems

1. **Unescaped quotes** inside the third snippet:

```jsonc
"    reporting.log_section_completion("X.Y.Z", "OK", checked=1, mismatched=0)",
```

Those inner double quotes need to be escaped (`\"`) or VS Code will choke.

2. A bit of inconsistent formatting / missing blank lines at the ends of bodies (not fatal, but we can clean it up).

3. You’ve got three related snippets:

   * `dmdbg` → defines `debug_module`
   * `dmuse` → uses `debug_module` from `toolkit.module_debug`
   * `debug_module` → another usage snippet, but the name collides with the function name and is a bit confusing

I’d keep all three, but rename the last one to something like `dmquick` or `dmreporting` so it’s obvious what it does when it pops up.

---

## Cleaned-up, ready-to-paste snippet file

Here’s a fixed and polished version of what you posted (safe to drop into `.code-snippets`):

```jsonc
{
  "debug_module_helper": {
    "prefix": "dmdbg",
    "description": "Debug + reload a module with optional smoke tests",
    "body": [
      "from importlib import reload",
      "import inspect",
      "",
      "def debug_module(mod, test_funcs=None):",
      "    \"\"\"",
      "    Quick helper to:",
      "      - reload a module",
      "      - print where it's loaded from",
      "      - optionally execute some test functions",
      "    \"\"\"",
      "    mod = reload(mod)",
      "    print(\"✅ Reloaded:\", mod.__name__)",
      "    print(\"   File   :\", getattr(mod, \"__file__\", \"<no file>\"))",
      "",
      "    if test_funcs:",
      "        for name, fn in test_funcs.items():",
      "            try:",
      "                print(f\"\\n— Running smoke test: {name}()\")",
      "                fn()",
      "                print(f\"   ✔ {name} ok\")",
      "            except Exception as e:",
      "                print(f\"   ⚠ {name} failed: {e}\")",
      "",
      "    return mod",
      ""
    ]
  },

  "debug_module_usage": {
    "prefix": "dmuse",
    "description": "Use debug_module to reload + smoke-test a module",
    "body": [
      "import ${1:telco_churn.utils.reporting} as mod_under_test",
      "from toolkit.module_debug import debug_module",
      "",
      "def _test_${2:log_section}():",
      "    mod_under_test.${2:log_section_completion}(\"X.Y.Z\", \"OK\", checked=1, mismatched=0)",
      "",
      "mod_under_test = debug_module(",
      "    mod_under_test,",
      "    test_funcs={\"${2:log_section}\": _test_${2:log_section}},",
      ")",
      ""
    ]
  },

  "debug_module_reporting_quick": {
    "prefix": "dmreporting",
    "description": "Quick debug_module harness for telco_churn.utils.reporting.log_section_completion",
    "body": [
      "import telco_churn.utils.reporting as reporting",
      "from toolkit.module_debug import debug_module  # adjust import if needed",
      "",
      "def _test_log_section():",
      "    reporting.log_section_completion(",
      "        \"X.Y.Z\",",
      "        \"OK\",",
      "        checked=1,",
      "        mismatched=0,",
      "    )",
      "",
      "reporting = debug_module(",
      "    reporting,",
      "    test_funcs={\"log_section_completion\": _test_log_section},",
      ")",
      ""
    ]
  }
}
```

### How these behave

* Type `dmdbg` → **defines** `debug_module` in the current file.
* Type `dmuse` → creates a **generic harness**:

  * lets you tab through:

    * the module import (`${1:telco_churn.utils.reporting}`)
    * the function name (`${2:log_section}` → `log_section_completion`)
* Type `dmreporting` → drops in a **ready-to-run** harness for `telco_churn.utils.reporting.log_section_completion` specifically.

---

### Tiny pro tip 💡💡

Since you now have a dedicated module:

```python
# toolkit/module_debug.py
def debug_module(...):
    ...
```

You’ll usually only need **`dmuse`** and **`dmreporting`** in new files.
`dmdbg` is there for when you’re in some totally new project / scratch file and want to bootstrap `debug_module` from scratch.

If you want, next step we can:

* add a `tktk` snippet for `from telco_churn import toolkit as tk`
* wire `debug_module`, `log_section_completion`, etc., into `telco_churn/toolkit/__init__.py`

so that your flow becomes:

1. Type `tktk` → you get `from telco_churn import toolkit as tk`
2. Type `tk.` → see **all your helpers** as autocomplete.


Short answer: you don’t need a special “fix my datetime” tool — you just need to make sure **the module that’s actually running** has the new imports and is reloaded. Right now, your notebook is still calling an *old in-memory version* of `log_section_completion` that doesn’t know about `datetime`.

Let’s walk it concretely.

---

## 1. Why you’re still getting `NameError: datetime`

The traceback:

```text
File .../telco_churn/utils/reporting.py:121, in log_section_completion(...)
    ts_utc = (
        datetime.now(timezone.utc)
NameError: name 'datetime' is not defined
```

That means: inside the *loaded* `telco_churn.utils.reporting` module, there is **no global name** `datetime`.

But your file **on disk** clearly has:

```python
from datetime import datetime, timezone
```

So one of these must be true:

1. The kernel is still using an **older version** of `reporting.py` that didn’t have that import when it was first imported, or
2. You have **two different `reporting.py` files** and Python is importing the other one, or
3. You shadowed `datetime` somewhere (less likely here, but possible).

Given what you’ve been doing, (1) is by far the most likely.

---

## 2. Sanity-check which code is actually running

In a fresh cell, run:

```python
import telco_churn.utils.reporting as reporting
from importlib import reload
import inspect

reporting = reload(reporting)

print("Module file:", reporting.__file__)
print("Has datetime import?", "datetime" in reporting.__dict__)
print("\n--- log_section_completion source ---")
print(inspect.getsource(reporting.log_section_completion))
```

You should see:

* `Module file:` pointing at
  `/Users/b/DATA/PROJECTS/Telco/Level_3/src/telco_churn/utils/reporting.py`
* In the source printout, `log_section_completion` should match the version you pasted (with `datetime.now(timezone.utc)` and the docstring).

If `datetime` is still missing from `__dict__` or the `getsource` body doesn’t match what you expect, then:

* Either the file wasn’t saved, or
* You’re editing a different file than the one being imported.

---

## 3. Make sure the function actually *prints* and *logs*

Right now your `log_section_completion` ends after building `record`:

```python
    if metrics:
        record["metrics"] = dict(metrics)
```

and then… it stops. In the old version you had:

* The console print:

```python
    parts = [f"✅ [{section}]", f"status={status}"]
    for k, v in (metrics or {}).items():
        parts.append(f"{k}={v}")
    print(" | ".join(parts))
```

* And the optional JSONL logging.

Make sure your **full** function looks like this (this is safe to paste over the current one):

```python
def log_section_completion(
    section: str,
    status: str,
    *,
    run_id: str | None = None,
    level: str = "info",
    log_dir: str | Path | None = None,
    log_name: str = "section2_runlog.jsonl",
    extra: Mapping[str, Any] | None = None,
    **metrics: Any,
) -> None:
    """
    Lightweight Section 2 logger.

    - Always prints a concise console line.
    - Optionally appends a JSONL record to a run log.
    """
    ts_utc = (
        datetime.now(timezone.utc)
        .isoformat(timespec="seconds")
        .replace("+00:00", "Z")
    )

    # Build record
    record: dict[str, Any] = {
        "timestamp_utc": ts_utc,
        "section": section,
        "status": status,
        "level": level,
    }
    if run_id is not None:
        record["run_id"] = run_id

    if extra:
        record.update(dict(extra))

    if metrics:
        record["metrics"] = dict(metrics)

    # ---- Console output (human friendly) -----------------------------------
    parts = [f"✅ [{section}]", f"status={status}"]
    for k, v in (metrics or {}).items():
        parts.append(f"{k}={v}")
    print(" | ".join(parts))

    # ---- Optional JSONL logging --------------------------------------------
    if log_dir is None:
        return

    log_dir = Path(log_dir)
    log_dir.mkdir(parents=True, exist_ok=True)
    log_path = log_dir / log_name

    try:
        with log_path.open("a", encoding="utf-8") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")
    except Exception as e:
        # We never want logging failures to break the notebook
        print(f"⚠️ log_section_completion: failed to write to {log_path}: {e}")
```

Then:

```python
from importlib import reload
import telco_churn.utils.reporting as reporting

reporting = reload(reporting)
reporting.log_section_completion("X.Y.Z", "OK", checked=1, mismatched=0)
```

If that prints your line, you’re good.

---

## 4. Do you need a dedicated “toolkit tool” for this?

You kind of **already have one**: your `debug_module` helper and/or this minimal pattern:

```python
from importlib import reload
import telco_churn.utils.reporting as reporting
import inspect

reporting = reload(reporting)
print("Module file:", reporting.__file__)
print(inspect.getsource(reporting.log_section_completion))
```

That *is* the tool:

* Reloads the module
* Shows you exactly what’s in it
* Confirms you’re using the file you think you’re using

If you wrap that in a reusable helper (`debug_module`) and a VS Code snippet (which you already started doing), you’ve effectively built your “fix weird import/module issues” wrench.

---

## 5. Where this fits in your mental model

* **`utils` folder** → reusable, project-specific **library** code (like `append_sec2`, `log_section_completion`)
* **`toolkit` folder** → your personal **dev-workflow helpers** (`debug_module`, grep tools, one-off sanity-checkers, etc.)

When something like this happens again:

1. Edit the module file.
2. Use your `debug_module` or quick reload snippet to ensure the new version is actually loaded.
3. Re-run your notebook cell that calls it.

If you want, next round we can design a tiny `tk.mod("telco_churn.utils.reporting")` helper that does the reload + print + one test in one line.
